# Converting geotagged-point tweets to GEOJSON

In [1]:
import pymongo, json, pprint, urllib.request, os.path, sys
import pandas as pd

In [2]:
config = json.load(open('tweets-on-a-map.config','r'))
pprint.pprint(config)

{'client': 'epic-analytics.cs.colorado.edu',
 'collection': 'tweets',
 'database': 'matthew',
 'img_root': 'http://epic-analytics.cs.colorado.edu:9000/jennings/infovis/map_images',
 'start_date': '2016-9-25',
 'web_root': '/data/www/jennings/infovis'}


In [3]:
client = pymongo.MongoClient('mongodb://'+config['client'])
db = client[config['database']]
tweets = db[config['collection']]
print("Found {0} tweets ".format(tweets.count()))

Found 13830253 tweets 


In [4]:
cursor = tweets.find({'geo' : {"$exists":True}}).batch_size(10)

In [5]:
start_date = pd.Timestamp(config['start_date'])

In [10]:
idx = 0;
feats = []
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    if os.path.exists("/data/www/jennings/infovis/map_images/thumb/"+tweet_id+".jpg"):
        jsonObj = { "type" : "Feature",
                 "geometry" : {"type" : "Point", "coordinates" : list(reversed(t['geo']['coordinates']))},
                 "properties" : {  "id" : tweet_id,
                                   "user" : t['actor']['preferredUsername'],
                                   "text" : t['body'],
                                   "day" : int( ( pd.Timestamp(t['postedTime']) - start_date ).total_seconds() /(3600*24) )
           }
         }
        feats.append(jsonObj)

In [11]:
with open('/data/www/jennings/infovis/geotagged-tweets.geojson','w') as outFile:
    json.dump({'type':"FeatureCollection","features":feats},outFile)